In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [9]:
from torch.autograd import Variable
from torchvision import datasets, transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn import functional as F

In [10]:

batch_size = 64
epochs = 10
input_dim = 784
output_dim = 10
lr_rate = 0.0001
momentum = 0.0 


In [11]:
# SVMModel
svm_model = nn.Linear(input_dim,output_dim)

In [17]:
train_data = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)
                   
test_data = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

subset_indices = ((train_data.train_labels == 0) + (train_data.train_labels == 1)).nonzero().view(-1)
train_loader = torch.utils.data.DataLoader(dataset=train_data, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           sampler=SubsetRandomSampler(subset_indices))

subset_indices = ((test_data.test_labels == 0) + (test_data.test_labels == 1)).nonzero().view(-1)
test_loader = torch.utils.data.DataLoader(dataset=test_data, 
                                          batch_size=batch_size,
                                          shuffle=False,
                                          sampler=SubsetRandomSampler(subset_indices))

In [18]:

svm_loss_criteria = SVM_Loss()
svm_optimizer = torch.optim.SGD(svm_model.parameters(), lr=lr_rate, momentum=momentum)

In [19]:
# Train the Model
total_step = len(train_loader)
for epoch in range(epochs):
    avg_loss_epoch = 0
    batch_loss = 0
    total_batches = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28)                      
        labels = Variable(2*(labels.float()-0.5))   
        outputs = svm_model(images)           
        loss_svm = svm_loss_criteria(outputs, labels)    
        svm_optimizer.zero_grad()
        loss_svm.backward()
        svm_optimizer.step()    
        
        
        total_batches += 1     
        batch_loss += loss_svm.item()

    avg_loss_epoch = batch_loss/total_batches
    print ('Epoch [{}/{}], Averge Loss:for epoch[{}, {:.4f}]' 
                   .format(epoch+1, epochs, epoch+1, avg_loss_epoch ))
        

# Calculate Accuracy
correct = 0.
total = 0.
for images, labels in test_loader:
    images = images.reshape(-1, 28*28)
    
    outputs = svm_model(images)    
    predicted = outputs.data >= 0
    total += labels.size(0) 
    correct += (predicted.view(-1).long() == labels).sum()  

In [20]:
print("lr_rate ", lr_rate)
print( "momentum", momentum)